In [1]:
from bs4 import BeautifulSoup
import re
import time
import mechanicalsoup
import random
import string
import requests
import json

In [2]:
class scrapper():
    def __init__(self, config):
        self.config = config
        self.browser = mechanicalsoup.StatefulBrowser()

    def login(self):
        self.browser.open(self.config['login'])
        self.browser.select_form(nr=0)
        self.browser['txtLogin'] = self.config['username']
        self.browser['txtPassword'] = self.config['password']
        self.browser.submit_selected()
        print("logging in")
        self.check_password()


    def api_post(self, link, data):
        r = requests.post(link, data=json.dumps(data))
        print(r)

    def password_generator(self, size=8, chars=string.ascii_lowercase + string.digits):
        return ''.join(random.choice(chars) for _ in range(size))

    def write_password(self, password):
        self.config["password"] = password

        with open("config.json", "w") as jsonFile:
            json.dump(self.config, jsonFile, indent=4, sort_keys=True)

    def check_password(self):
    
        page = self.browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')
        #print(soup)
        # if this is the change password page
        if soup.title.text == '\r\n\tChange Password\r\n':
            # generate new password
            new_password = self.password_generator()
            # save to file
            self.write_password(new_password)
            self.browser.select_form(nr=0)
            self.browser['txtLoginID'] = self.config['username']
            self.browser['txtCurrPass'] = self.config['password']
            self.browser['txtNewPass'] = new_password
            self.browser['txtConfirmPass'] = new_password
            self.browser.submit_selected()
            print('password updated to ', new_password)

In [3]:
#this is for rich.co.ke
class rich_data(scrapper):
    def __init__(self, c):
        super().__init__(c)
        print("rich scapper initialized")
        
    def crawl_data(self):
        browser = browser = mechanicalsoup.StatefulBrowser()
        #use config for rich data
        browser.open("http://www.rich.co.ke/rcdata/nsestocks.php")
        page = browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')

        #skip straight to the table we want and pull all the rows
        rich_data = soup.body.find_all('table')[16].find_all("tr")

        #remove the first row cause its just the heading
        rich_data.pop(0)
        final_data = []

        #add the data to a list as text
        for tr in rich_data:
            td = tr.find_all("td")
            final_data.append({"stock":str(td[1].get_text()),#name of company
                               "previous":str(td[2].get_text()),
                               "current":str(td[3].get_text()),
                               "percentage_change":str(td[4].get_text()),
                               "volume":str(td[6].get_text())
                              })
        return final_data


In [4]:
class wazua_data(scrapper):
    def __init__(self, c):
        super().__init__(c)
        print("wazua scapper initialized")
    def crawl_data(self):
        browser = browser = mechanicalsoup.StatefulBrowser()
        browser.open("http://www.wazua.co.ke/investor/latestprices.aspx")
        page = browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')

        #skip straight to table we need
        data = soup.body.find_all('table')[37].find_all("tr")

        #remove the first row cause its just the heading
        data.pop(0)
        final_data = []

        #somethimg
        for tr in data:
            td = tr.find_all("td")

            #add the data to a list as text and remove \n characters with replace()
            final_data.append({"title":str(td[0].get_text().replace("\n","")),
                               "price":str(td[1].get_text().replace("\n","")),
                               "volume":str(td[2].get_text().replace("\n","")),

                               #hopefully they'll add some more useful info than this
                               #str(td[3].get_text()),
                               #str(td[4].get_text())
                              })
        return final_data


In [5]:
market_sdarr = []


In [6]:


class ticker_data(scrapper):
    def __init__(self, c):
        super().__init__(c)
        print("ticker scapper initialized")

        
    def crawl_data(self):
        self.login()
        self.browser.open(self.config['ticker'])
        page = self.browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')
        
        tickers=[]
        if(soup.find("div", {"class": "CSPLTickerContainer"})):
            div = soup.find("div", {"class": "CSPLTickerContainer"})
            ul =  div.find("ul")
            li =  ul.find("li")
            

            for li in ul:
                combo = li.text.split('|', 1)[0]
                security_price, change = combo.split(' ')
                security = re.split('(\d+)',security_price)
                price = ''.join(security[1:4])
                #items.append([security[0] ,price , change])
                tickers.append({'security': security[0],
                                'price': price,
                                'price_change': change})

        return tickers

In [19]:
class market_data(scrapper):
    def __init__(self, c):
        super().__init__(c)
        print("market scapper initialized")

        
    def crawl_data(self):
        self.login()
        self.browser.open(self.config['market'])
        page = self.browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')
        
        markets=[]
        
        script = soup.find_all("script")
        if(script[22]):
            jss =  str(script[22])
            str_replace_qoutes = jss.replace('\"', '\\"')
            str_minus_semi_colon = str_replace_qoutes.replace(";"," ")
            str_minus_sdarr = str_minus_semi_colon.replace("sDArr","")
            str_minus_equal = str_minus_sdarr.replace("=","")

            #everything before sdarr
            head, sep, tail = str_minus_equal.partition('(jsiScripsInMW,50) ')
            #everything after sdarr
            head2, sep2, tail2 = tail.partition('  jsiMWrefreshDelay')
            result = re.sub("[\(\[].*?[\)\]]", "", head2)
            arr = result.split("' '")
            arr[0] = arr[0].replace("\'","")
            arr[-1] = arr[-1].replace("\'","")

            sdarr = []
            sec = []
            count =1
            for i in range(len(arr)):
                count = count + 1
                sec.append(arr[i])
                if count ==22:
                    count = 1
                    sdarr.append(sec)
                    sec =[]
            for row in sdarr:
                sub_row = [row[2], row[8], row[9], row[10], row[11], row[12]]
                high_low = self.sortlist(sub_row)
                markets.append({"security": row[0],
                                "last_price": row[2],
                                "demand_qty": row[5],
                                "demand_price": row[6],
                                "supply_price": row[8],
                                "supply_qty": row[7],
                                "last_qty": row[4],
                                "high": max(high_low),
                                "low": min(high_low)
                                })
            
        return markets, sdarr

    def sortlist(self, mylist):
        newlist=[]
        for item in mylist:
            if float(item) > 0:
                newlist.append(item)

        if newlist == []:
            newlist = [str(0.00), str(0.00)]
        return newlist
    


In [20]:
# # from ticker_data import ticker_data
# # from market_data import market_data
# import pytz
# import time as tt
# from datetime import datetime, timedelta, date, time
# import json

# #check if its a weekday
# def is_week_day():
#     return datetime.today().weekday() < 5

# #check if it's correct time
# def is_in_time():
#     tz = pytz.timezone('Africa/Nairobi')
#     nairobi_now = datetime.now(tz).time()
#     nairobi_hour = nairobi_now.hour
#     nairobi_minute = nairobi_now.minute
#     set_time = time(3, 35)
#     time_hour = set_time.hour
#     time_minute = set_time.minute

#     if nairobi_hour == time_hour and nairobi_minute == time_minute:
#         return 1
#     return 0




# print("crawler active")
# while 1:
#     #if its weekday and correct time then crawl site
#     if is_week_day() and is_in_time():
#         f = open('config.json')
#         #load configuration file
#         config = json.load(f)
#         t = ticker_data(config)
#         m = market_data(config)
#         w = wazua_data(config)
#         r = rich_data(config)
#         collected = 0
#         for s in (m, t):
#             s.login()
#             collected = s.crawl_data()
#             print(collected)
#         if collected == 1:
#             print("[",datetime.now()," ] info collected  ")
#             print()

#             tt.sleep(300)
#         else:
#             print("[", datetime.now(), " ] failed ")
#             print()


In [24]:
import json
#if we could use maps, this would be much more efficient
data_list = []
f = open('config_dev.json')
config = json.load(f)
t = ticker_data(config)
m = market_data(config)
#w = wazua_data(config)
r = rich_data(config)

#wazua = w.crawl_data()#[[0,1,34,566,677], [1,2,345,6785,89]]
rich = r.crawl_data()#[[0,3242,1,4353,786], [1,23432,2,345,45]]
ticker= t.crawl_data()#[[0,234,5756,1], [1,567567,2343,2]]
market,m_sdarr = m.crawl_data()#[[0,345,6786,24,1], [1,24234,6456,87,2]]

#data_list.append(wazua)
data_list.append(rich)
data_list.append(ticker)
data_list.append(market)

ticker scapper initialized
market scapper initialized
rich scapper initialized
logging in
logging in


In [26]:
#print(data_list[0][4])
#print()
print(data_list[0][6])
print()
print(data_list[1][16])
print()
print(data_list[2][26])

{'percentage_change': '1.72%', 'volume': '17,500', 'current': '0.59', 'stock': 'Home Afrika Ltd', 'previous': '0.58'}

{'price': '0.57', 'price_change': '-0.02', 'security': 'HAFR'}

{'supply_qty': '0', 'demand_price': '0.00', 'last_qty': '500', 'high': '8.50', 'demand_qty': '0', 'low': '8.42', 'last_price': '8.50', 'supply_price': '0.00', 'security': 'FAHR'}


In [12]:
stock_list=[0,1]
final_data=[]

for j in range(len(stock_list)):
    final_data.append([data_list[1][j][0],
                       data_list[2][j][0]])
                       data_list[3][j][3],
                       #data_list[3][j][4])
                     



IndentationError: unexpected indent (<ipython-input-12-ad37fa3c7395>, line 7)

In [13]:
final_data

NameError: name 'final_data' is not defined

['"ACCS":{},\n\t"ADSS":{},\n\t"BAMB":{},\n\t"BAT":{},\n\t"BAUM":{},\n\t"BBK":{},\n\t"BERG":{},\n\t"BOC":{},\n\t"BRIT":{},\n\t"CABL":{},\n\t"CARB":{},\n\t"CFC":{},\n\t"CFCI":{},\n\t"CG":{},\n\t"CIC":{},\n\t"CITY":{},\n\t"CMC":{},\n\t"COOP":{},\n\t"DTK":{},\n\t"ARM":{},\n\t"DTKR":{},\n\t"EABL":{},\n\t"EGAD":{},\n\t"EQT":{},\n\t"EVRD":{},\n\t"FIRE":{},\n\t"FTGH":{},\n\t"HAFR":{},\n\t"HBL":{},\n\t"HFCK":{},\n\t"I&M":{},\n\t"ICDC":{},\n\t"JUB":{},\n\t"KAPC":{},\n\t"KCB":{},\n\t"KEGN":{},\n\t"KENO":{},\n\t"KNRE":{},\n\t"KPLC":{},\n\t"KQ":{},\n\t"KUKZ":{},\n\t"KURV":{},\n\t"LIMT":{},\n\t"LKL":{},\n\t"MASH":{},\n\t"MSC":{},\n\t"NBK":{},\n\t"NIC":{},\n\t"NICR":{},\n\t"NMG":{},\n\t"NSE":{},\n\t"OCH":{},\n\t"OCHR":{},\n\t"ORCH":{},\n\t"PAFR":{},\n\t"PORT":{},\n\t"REA":{},\n\t"SASN":{},\n\t"SCAN":{},\n\t"SCBK":{},\n\t"SCOM":{},\n\t"SGL":{},\n\t"TCL":{},\n\t"TOTL":{},\n\t"TPS":{},\n\t"UCHM":{},\n\t"UNGA":{},\n\t"UTK":{},\n\t"WTK":{},\n\t"XPRS":{},\n\nNotice:  Trying to get property \'id\' of non-ob

In [39]:
#companies



for company in cba:
    all_companies.append(company)
for company in wazua:
    all_companies.append(company[0])
for company in rich:
    all_companies.append(company[0])

all_companies.sort()

for company in all_companies:
    json_file = json_file + "\""+company + "\": \"\",\n"
print(json_file)


NameError: name 'wazua' is not defined

In [16]:
f = open('response.txt','r')
response = f.read()
response = response.replace("xdrCall.JsonXSHR(","")
response = response.split("^^||^", 1)[0]
response = response.split("1^^", 1)[1]
response = response.split("|")
stocks = []
for stock in response:
    stocks.append(stock.split("#"))

In [35]:
len(ticker)

47

In [36]:
m_sdarr[64]

['SCOM',
 '',
 '28.80',
 '14:58:33',
 '3800',
 '0',
 '0.00',
 '0',
 '0.00',
 '29.10',
 '29.10',
 '28.75',
 '29.00',
 '100',
 '0',
 '10.00',
 '10.00',
 'SAFARICOM LTD',
 '0.00',
 '220300.00',
 '53']

In [51]:
all_companies = []

for stock in m_sdarr:
    all_companies[stock[0]]=[]

TypeError: list indices must be integers or slices, not str

In [49]:
all_companies["SCOM"]

TypeError: list indices must be integers or slices, not str